<a href="https://colab.research.google.com/github/nisha1365/GEN-AI-POC/blob/main/tweet_with_sentiment_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# Load the Excel file
file_path = "/content/tweets_by_topics_110_final.xlsx"
xls = pd.ExcelFile(file_path)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# Create a function to clean the text

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import re
def clean_text(text):
    # Remove mentions (@)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    # Remove hashtags (#) without removing the word following it
    text = re.sub(r'#(\w+)', r'\1', text)
    # Remove URLs (http/https)
    text = re.sub(r'https?://\S+|www\.\S+|https?\/\/t\.co\/\S+', '', text)
    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" # emoticons
                               u"\U0001F300-\U0001F5FF" # symbols & pictographs
                               u"\U0001F680-\U0001F6FF" # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                               u"\U00002500-\U00002BEF" # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f" # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text = re.sub(emoji_pattern, '', text)
    # Remove extra white spaces
    text = re.sub(r'\s+', ' ', text.strip())
    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into text
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Create a sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Create a new Excel writer object
new_file_path = "/content/tweets_with_sentiment.xlsx"
writer = pd.ExcelWriter(new_file_path)

# Iterate through each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Add a new column for cleaned text
    df['clean_text'] = df['text'].apply(clean_text)

    # Add a new column for sentiment label
    df['sentiment_label'] = df['clean_text'].apply(lambda x: sid.polarity_scores(x)['compound'])

    # Classify the sentiment based on the compound score
    df['sentiment_label'] = df['sentiment_label'].apply(lambda score: 'positive' if score > 0 else ('negative' if score < 0 else 'neutral'))

    # Save the modified DataFrame to the new Excel file
    df.to_excel(writer, sheet_name=sheet_name, index=False)

# Close the Excel writer
writer.save()

print("Sentiment analysis and data cleaning completed. New file created:", new_file_path)

<ipython-input-7-f3c25fff2f96>:26: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Sentiment analysis and data cleaning completed. New file created: /content/tweets_with_sentiment.xlsx
